In [1]:
#Import CSV
import csv

#Read order requirements/rod requirements
with open('/Users/btf/Documents/15.066 Sys Optimization/MiniProject/input.csv') as f:
	rod_specs = list(csv.reader(f,quoting=csv.QUOTE_NONNUMERIC))

In [5]:
beam_length = 12005
cut_waste = 5
beam_cost = 1380
weld_cost = 156

rod_orders = {}
for rod in rod_specs:
    rod_orders[int(rod[0])] = rod[1]
print(rod_orders)


{1: 11962.55, 2: 11962.55, 3: 11962.55, 4: 11962.55, 5: 11962.55, 6: 11962.55, 7: 11953.0, 8: 11953.0, 9: 11953.0, 10: 11953.0, 11: 11953.0, 12: 11953.0, 13: 11953.0, 14: 11953.0, 15: 11953.0, 16: 11953.0, 17: 11953.0, 18: 11953.0, 19: 11953.0, 20: 11953.0, 21: 11817.01, 22: 11817.01, 23: 11817.01, 24: 11817.01, 25: 11817.01, 26: 11817.01, 27: 11817.01, 28: 11817.01, 29: 11817.01, 30: 11817.01, 31: 11817.01, 32: 11817.01, 33: 11817.01, 34: 11817.01, 35: 11774.01, 36: 11774.01, 37: 11774.01, 38: 11764.12, 39: 11764.12, 40: 11764.12, 41: 11764.12, 42: 11764.12, 43: 11764.12, 44: 11752.51, 45: 11752.51, 46: 11752.51, 47: 11583.02, 48: 11583.02, 49: 11583.02, 50: 11575.69, 51: 11575.69, 52: 11575.69, 53: 11575.68, 54: 11575.68, 55: 11575.68, 56: 11465.8, 57: 11465.8, 58: 11465.8, 59: 11465.8, 60: 11465.8, 61: 11465.8, 62: 11465.8, 63: 11465.8, 64: 11465.8, 65: 11465.8, 66: 11465.8, 67: 11465.8, 68: 11465.8, 69: 11465.8, 70: 11376.7, 71: 11376.68, 72: 11376.68, 73: 11376.68, 74: 11376.68, 7

In [6]:
'''rod_orders[:3]
{1: 11962.55, 2: 11962.55, 3: 11962.55,
'''
beams = {}
for k in range(1,252):
    beams[k] = []

n = 1
rods_used  = list()
beam_waste = 0
beam_waste_log = []
for beam in beams:
    
    beam_rem = beam_length
  
    for rod in sorted(rod_orders):
        if beam_rem - (rod_orders[rod]) == 0 and rod not in rods_used: 
            beams[beam] += [rod]
            rods_used.append(rod)
        elif beam_rem - (rod_orders[rod]+cut_waste) >= 0 and rod not in rods_used:
            beams[beam] += [rod]
            beam_rem = beam_rem - rod_orders[rod]+cut_waste
            rod_max = rod
            rods_used.append(rod)
           
    beam_waste += beam_rem
    beam_waste_log += [beam_rem]



           


In [7]:
print('beams', beams, 'beams')

print(beam_waste)
print(beam_waste_log)
print(sum(beam_waste_log))

#print('rod_orders', rod_orders, 'rod_orders')

#print('rods_used', rods_used, 'rods_used')

beams {1: [1], 2: [2], 3: [3], 4: [4], 5: [5], 6: [6], 7: [7], 8: [8], 9: [9], 10: [10], 11: [11], 12: [12], 13: [13], 14: [14], 15: [15], 16: [16], 17: [17], 18: [18], 19: [19], 20: [20], 21: [21], 22: [22], 23: [23], 24: [24], 25: [25], 26: [26], 27: [27], 28: [28], 29: [29], 30: [30], 31: [31], 32: [32], 33: [33], 34: [34], 35: [35], 36: [36], 37: [37], 38: [38, 393], 39: [39, 394], 40: [40, 395], 41: [41, 396], 42: [42, 397], 43: [43, 398], 44: [44], 45: [45], 46: [46], 47: [47], 48: [48], 49: [49], 50: [50], 51: [51], 52: [52], 53: [53], 54: [54], 55: [55], 56: [56], 57: [57], 58: [58], 59: [59], 60: [60], 61: [61], 62: [62], 63: [63], 64: [64], 65: [65], 66: [66], 67: [67], 68: [68], 69: [69], 70: [70], 71: [71], 72: [72], 73: [73], 74: [74], 75: [75], 76: [76], 77: [77], 78: [78], 79: [79], 80: [80], 81: [81], 82: [82], 83: [83], 84: [84], 85: [85], 86: [86], 87: [87], 88: [88], 89: [89], 90: [90], 91: [91], 92: [92], 93: [93], 94: [94], 95: [95], 96: [96], 97: [97], 98: [98], 9

In [8]:
import gurobipy as gp

In [13]:
#Gurobi
m = gp.Model("A")

beam_length = 12005
cut_waste = 5
beam_cost = 1380
weld_cost = 156

rod_orders = {}
for rod in rod_specs:
    rod_orders[int(rod[0])] = rod[1]
beams = {}
for k in range(1,252):
    beams[k] = []

rod_beam = {}

for rod in rod_orders:
    for beam in beams:
        rod_beam[(rod, beam)] = m.addVar(name = f'R{rod}B{beam}', vtype=gp.GRB.BINARY)

m.setObjective(sum(beam_cost*rod_beam[(rod, beam)] for rod in rod_orders for beam in beams), gp.GRB.MINIMIZE)

for rod in rod_orders:
    m.addConstr(sum(rod_beam[(rod, beam)] for beam in beams) == 1)
for beam in beams:
    m.addConstr(sum(rod_orders[rod]*rod_beam[(rod, beam)]+cut_waste*rod_beam[(rod, beam)] for rod in rod_orders) <= beam_length)
m.optimize()
m.write('results.attr')

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (mac64[arm])
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads
Optimize a model with 649 rows, 99898 columns and 199796 nonzeros
Model fingerprint: 0x09ac770d
Variable types: 0 continuous, 99898 integer (99898 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+04]
  Objective range  [1e+03, 1e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+04]
Found heuristic solution: objective 549240.00000
Presolve time: 0.20s
Presolved: 649 rows, 99898 columns, 199796 nonzeros
Variable types: 0 continuous, 99898 integer (99898 binary)

Explored 0 nodes (0 simplex iterations) in 0.23 seconds (0.59 work units)
Thread count was 10 (of 10 available processors)

Solution count 1: 549240 

Optimal solution found (tolerance 1.00e-04)
Best objective 5.492400000000e+05, best bound 5.492400000000e+05, gap 0.0000%


In [22]:
beams_lp = {}
for k in range(1,252):
    beams_lp[k] = []

for rod in rod_beam:
    if rod_beam[rod].x == 1:
        beams_lp[rod[1]] += [rod[0]]

print(beams_lp)


{1: [99], 2: [114], 3: [55], 4: [58], 5: [11], 6: [245, 284, 350], 7: [182, 305], 8: [218, 259], 9: [113], 10: [237, 316, 381], 11: [38], 12: [48], 13: [196, 306], 14: [233, 265, 345], 15: [103, 395], 16: [161, 351], 17: [247, 285, 375], 18: [205, 382, 385], 19: [12], 20: [221, 264], 21: [140, 396], 22: [49], 23: [75], 24: [223, 279, 391], 25: [51], 26: [72], 27: [156, 358], 28: [158, 390], 29: [80], 30: [21], 31: [84], 32: [100], 33: [230, 289, 362], 34: [46], 35: [137], 36: [174, 320], 37: [240, 322, 346], 38: [172, 379], 39: [18], 40: [22], 41: [193, 354, 374], 42: [239, 311, 315], 43: [14], 44: [117], 45: [101], 46: [219, 295], 47: [141], 48: [116], 49: [81], 50: [248, 261, 338], 51: [235, 321, 327], 52: [139], 53: [152, 326], 54: [76], 55: [53], 56: [169, 376], 57: [135], 58: [28], 59: [138], 60: [37], 61: [86], 62: [144], 63: [73], 64: [79], 65: [199, 252], 66: [179, 271], 67: [214, 294], 68: [63], 69: [54], 70: [225, 278, 325], 71: [92], 72: [127], 73: [67], 74: [243, 331, 339],

In [10]:
# after optimizing, write to output file
with open('output.csv', mode='w', newline='') as f:
	writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_NONNUMERIC)

	# loop over all rods
	for rod in range(1,399):	
		# first column is rod number
		rod_length_list = [rod]
		# loop through all beams to see if a length is used for this rod
		for beam in rods:	
			rod_length_list += rod_orders[rod]*[rod_beam[rod,beam].x] # substitute your decision variable for ROD_ASSIGNMENT_VARIABLE

		# write row to file
		writer.writerow(rod_length_list)

NameError: name 'rods' is not defined